In [1]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.ensemble
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [3]:
trainset = pd.read_csv('train.csv')
testset = pd.read_csv('test.csv')
trainnames = trainset['smiles']
train = trainset.drop('smiles',1)
testnames = testset['smiles']
testId = testset['Id']
test = testset.drop(['smiles','Id'],1)
ytrain = train['gap']
xtrain = train.drop('gap',1)

In [5]:
print train.shape
print test.shape

(1000000, 257)
(824230, 256)


In [7]:
# i = 0
# while i < train.shape[0]:
#     smiles_train[i] = trainnames[i:i+10000]
#     i = i+10000
# if 
n=0
smiles_train = []
for i in range(10):
    cut = trainnames[n:n+100000]
    smiles_train.append(cut)
    print "Made cut ", i

Made cut  0
Made cut  1
Made cut  2
Made cut  3
Made cut  4
Made cut  5
Made cut  6
Made cut  7
Made cut  8
Made cut  9
Made cut  10
Made cut  11
Made cut  12
Made cut  13
Made cut  14
Made cut  15
Made cut  16
Made cut  17
Made cut  18
Made cut  19
Made cut  20
Made cut  21
Made cut  22
Made cut  23
Made cut  24
Made cut  25
Made cut  26
Made cut  27
Made cut  28
Made cut  29
Made cut  30
Made cut  31
Made cut  32
Made cut  33
Made cut  34
Made cut  35
Made cut  36
Made cut  37
Made cut  38
Made cut  39
Made cut  40
Made cut  41
Made cut  42
Made cut  43
Made cut  44
Made cut  45
Made cut  46
Made cut  47
Made cut  48
Made cut  49
Made cut  50
Made cut  51
Made cut  52
Made cut  53
Made cut  54
Made cut  55
Made cut  56
Made cut  57
Made cut  58
Made cut  59
Made cut  60
Made cut  61
Made cut  62
Made cut  63
Made cut  64
Made cut  65
Made cut  66
Made cut  67
Made cut  68
Made cut  69
Made cut  70
Made cut  71
Made cut  72
Made cut  73
Made cut  74
Made cut  75
Made cut  76
Made cut 

In [ ]:
smiles_test = testnames[:10000]

smally = ytrain[:10000]
validy = ytrain[10000:20000]
smalltest = test[:10000]
print smally.shape
print validy.shape
print smalltest.shape

In [12]:
mols = []
for smile in trainnames:
    #print "Name: ", smile
    m = Chem.MolFromSmiles(smile)
    #print "Molecule: ",m
    mols.append(m)
    #print "List length: ",len(mols)
print "Mols found."

 Name:  c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2nsnc12
Molecule:  <rdkit.Chem.rdchem.Mol object at 0x1235377c0>
Name:  C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[SiH2]C=c12
Molecule:  <rdkit.Chem.rdchem.Mol object at 0x123537750>
Name:  [nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-23)c2ccccc12
Molecule:  <rdkit.Chem.rdchem.Mol object at 0x123537590>
Name:  [nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13)c1=C[SiH2]C=c21
Molecule:  <rdkit.Chem.rdchem.Mol object at 0x123537830>
Name:  c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1
Molecule:  <rdkit.Chem.rdchem.Mol object at 0x1235378a0>
Name:  C1=Cc2cnc3cc4cc(-c5scc6[nH]ccc56)c5ccccc5c4cc3c2[SiH2]1
Molecule:  <rdkit.Chem.rdchem.Mol object at 0x123537910>
Name:  c1ncc(s1)-c1cnc2c(c1)oc1c2ccc2ccccc12
Molecule:  <rdkit.Chem.rdchem.Mol object at 0x123537980>
Name:  c1sc(-c2ccc3c(c2)sc2c3c3=CCC=c3c3cccnc23)c2[se]ccc12
Molecule:  <rdkit.Chem.rdchem.Mol object at 0x1235379f0>
Name:  c1ccc(o1)-c1cc2cc3cc4c5c[nH]cc5ccc4cc3cc2o1
Molecule:  <rd

In [13]:
smile_train = []
for i in range(10):
    cut = trainnames[i*100000:(i+1)*100000]
    smile_train.append(cut)
    print "Appended train cut ", i
    pd.DataFrame(smile_train).to_csv("feature_transorm" + str(i), index=None)

j = 0  
smile_test = []
while (j+1)*100000 < test.shape[0]:
    cut = testnames[j*100000:(j+1)*100000]
    smile_test.append(cut)
    j=j+1
    print "Appended test cut ", j
    pd.DataFrame(smile_test).to_csv("feature_transorm" + str(j), index=None)
final = testnames[j*100000:(testnames.shape[0]-j*100000)]
smile_test.append(final)
pd.DataFrame(smile_test).to_csv("feature_transorm_test" + str(j+1), index=None)

print "Appended final test cut."

Appended train cut  0
Appended train cut  1
Appended train cut  2
Appended train cut  3
Appended train cut  4
Appended train cut  5
Appended train cut  6
Appended train cut  7
Appended train cut  8
Appended train cut  9
Appended test cut  1
Appended test cut  2
Appended test cut  3
Appended test cut  4
Appended test cut  5
Appended test cut  6
Appended test cut  7
Appended test cut  8
Appended final test cut.


In [14]:
smile_train

[0        c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...
 1        C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...
 2        [nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...
 3        [nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...
 4           c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1
 5        C1=Cc2cnc3cc4cc(-c5scc6[nH]ccc56)c5ccccc5c4cc3...
 6                    c1ncc(s1)-c1cnc2c(c1)oc1c2ccc2ccccc12
 7        c1sc(-c2ccc3c(c2)sc2c3c3=CCC=c3c3cccnc23)c2[se...
 8              c1ccc(o1)-c1cc2cc3cc4c5c[nH]cc5ccc4cc3cc2o1
 9          [nH]1ccc2c3c[nH]cc3c3cc(-c4cncs4)c4=CCC=c4c3c12
 10       [nH]1c(cc2c3cocc3c3c(ccc4ccc5=CCC=c5c34)c12)-c...
 11             c1cc2oc3c(sc4cc([se]c34)-c3cncc4nsnc34)c2o1
 12                  [nH]1c(cc2cnc3cc4ccoc4cc3c12)-c1ccccc1
 13           [nH]1ccc2ccc3c4ncc(cc4[nH]c3c12)-c1scc2occc12
 14       c1sc(-c2sc(-c3sc(-c4ncncn4)c4nccnc34)c3cc[nH]c...
 15          c1cc2ncc(cc2s1)-c1cc2c(ccc3ccccc23)c2c[nH]cc12
 16             c1ccc(-c2cc3oc4ccc5c[nH]

In [15]:
mols_train = []
for smiles in smile_train:
    chunk = []
    for smile in smiles:
        m = Chem.MolFromSmiles(smile)
        chunk.append(m)
    mols_train.append(chunk)


KeyboardInterrupt: 

In [ ]:
mols_test = []
for smiles in smile_test:
    chunk = []
    for smile in smiles:
        m = Chem.MolFromSmiles(smile)
        chunk.append(m)
    mols_test.append(chunk)